### 欠測データへの対処

* ほとんどの計算ツールは欠測値に対処できないか、予期せぬ動作をする。
* 下記が欠測データの例

In [19]:
import pandas as pd
from io import StringIO

print('---欠測値を含むデータ---')
csv_data = '''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
print(df)

print('---欠測値のカウント---')
print(df.isnull().sum())

---欠測値を含むデータ---
      A     B     C    D
0   1.0   2.0   3.0  4.0
1   5.0   6.0   NaN  8.0
2  10.0  11.0  12.0  NaN
---欠測値のカウント---
A    0
B    0
C    1
D    1
dtype: int64


### 欠測値を持つサンプル、特徴量を取り除く

* 下記のように欠測値をもつ行や列を削除できる。
* 削除しすぎると有益な情報が抜けてしまう場合がある。

In [21]:
print('---欠測値を含む行を削除---')
print(df.dropna())

print('\n---欠測値を含む列を削除---')
print(df.dropna(axis=1))

---欠測値を含む行を削除---
     A    B    C    D
0  1.0  2.0  3.0  4.0

---欠測値を含む列を削除---
      A     B
0   1.0   2.0
1   5.0   6.0
2  10.0  11.0


### 欠測値を補完する

* 欠測値を列全体と置き換える

In [30]:
from sklearn.preprocessing import Imputer
# storategyはmean(平均値)の他にmedian(中央値)とmost_frequent(最頻値)が利用可能
imr = Imputer(missing_values='NaN', strategy='mean', axis=0)

imr = imr.fit(df)

imputed_data = imr.transform(df.values)
print('列の平均値で補完')
print(imputed_data)

imr = Imputer(missing_values='NaN', strategy='mean', axis=1)
imr = imr.fit(df)
imputed_data = imr.transform(df.values)
print('\n行の平均値で補完')
print(imputed_data)

列の平均値で補完
[[  1.    2.    3.    4. ]
 [  5.    6.    7.5   8. ]
 [ 10.   11.   12.    6. ]]

行の平均値で補完
[[  1.           2.           3.           4.        ]
 [  5.           6.           6.33333333   8.        ]
 [ 10.          11.          12.          11.        ]]


## カテゴリデータの処理

* 服のサイズ（S,M,L）などのカテゴリを表すデータのこと。
* 名義と順序の特徴量を分ける必要がある。服のサイズは順序特徴量(S<M<L)だが、色は名義特徴量。

In [52]:
df = pd.DataFrame([
    ['green', 'M', 10.1, 'class1'],
    ['red', 'L', 13.5, 'class2'],
    ['blue', 'XL', 15.3, 'class1'],
])

df.columns = ['色', 'サイズ', '値段', 'クラス']
df

,色,サイズ,値段,クラス
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


#### 順序特徴量のマッピング

* カテゴリ文字列を整数化して推定器が扱えるようにする。

In [53]:
size_mapping = {'XL': 3, 'L':2, 'M':1}

df['サイズ'] = df['サイズ'].map(size_mapping)
df

,色,サイズ,値段,クラス
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


#### クラスラベルのエンコーディング

scikit-learnの内部で変換させるのではなく明示的に変換させるのが良い。

In [54]:
import numpy as np
class_mapping = {label:idx for idx, label in enumerate(np.unique(df['クラス']))}
print(class_mapping)

df['クラス'] = df['クラス'].map(class_mapping)
df

{'class1': 0, 'class2': 1}


,色,サイズ,値段,クラス
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [55]:
# 元に戻す
inv_class_mapping = {v:k for k, v in class_mapping.items()}
print(inv_class_mapping)
df['クラス'] = df['クラス'].map(inv_class_mapping)
df

{0: 'class1', 1: 'class2'}


,色,サイズ,値段,クラス
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [57]:
from sklearn.preprocessing import LabelEncoder
# ラベルエンコーダで自動的に数値化してくれる
class_le = LabelEncoder()
y = class_le.fit_transform(df['クラス'].values)
print(y)
# 元に戻す
class_le.inverse_transform(y)

[0 1 0]


array(['class1', 'class2', 'class1'], dtype=object)

### 名義特徴量でのone-hotエンコーディング